In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

train_df = pd.read_csv('/content/train.csv')
X = train_df.drop('label', axis=1)
y = train_df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=50)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"Explained Variance by PCA: {sum(pca.explained_variance_ratio_)*100:.2f}%")


classifier = KNeighborsClassifier(n_neighbors=10, metric='minkowski', p=2, weights='distance')
classifier.fit(X_train_pca, y_train)
y_pred = classifier.predict(X_test_pca)
print(f"Model Accuracy for KNN Model with PCA: {accuracy_score(y_pred, y_test)*100:.2f}%")

svm_model = SVC(kernel='linear', max_iter=100)
svm_model.fit(X_train_pca, y_train)
y_val_pred = svm_model.predict(X_test_pca)
svm_acc = accuracy_score(y_val_pred, y_test)
print(f'SVM Validation Accuracy with PCA: {svm_acc:.4f}')

svm_model_2 = SVC(kernel='rbf', max_iter=100)
svm_model_2.fit(X_train_pca, y_train)
pred_y = svm_model_2.predict(X_test_pca)
svm_acc2 = accuracy_score(pred_y, y_test)
print(f'SVM Validation Accuracy with RBF and PCA: {svm_acc2:.4f}')
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)
nn_model = Sequential([
    Dense(128, input_dim=X_train_pca.shape[1], activation='relu'),
    Dense(64, activation='relu'),
    Dense(y_train_encoded.shape[1], activation='softmax')
])
nn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = nn_model.fit(X_train_pca, y_train_encoded, epochs=10, batch_size=32, verbose=1)
loss, nn_acc = nn_model.evaluate(X_test_pca, y_test_encoded, verbose=0)
print(f'Neural Network Model Accuracy with PCA: {nn_acc*100:.2f}%')


Explained Variance by PCA: 56.15%
Model Accuracy for KNN Model with PCA: 95.70%


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM Validation Accuracy with PCA: 0.5399


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM Validation Accuracy with RBF and PCA: 0.8783


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.8061 - loss: 0.6648
Epoch 2/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9494 - loss: 0.1665
Epoch 3/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9641 - loss: 0.1127
Epoch 4/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9705 - loss: 0.0938
Epoch 5/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9745 - loss: 0.0784
Epoch 6/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9791 - loss: 0.0689
Epoch 7/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9819 - loss: 0.0567
Epoch 8/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9853 - loss: 0.0472
Epoch 9/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9870 - loss: 0.0411
Epoch 10/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9862 - loss: 0.0399
Neural Network Model Accuracy with PCA: 96.76%
